In [2]:
import pandas as pd

categories_df = pd.read_csv('/Users/sunvidaneja/Downloads/DS_NLP_search_data/categories.csv')
brand_category_df = pd.read_csv('/Users/sunvidaneja/Downloads/DS_NLP_search_data/brand_category.csv')
offer_retailer_df = pd.read_csv('/Users/sunvidaneja/Downloads/DS_NLP_search_data/offer_retailer.csv')

merged_brand_category = brand_category_df.merge(
    categories_df,
    left_on='BRAND_BELONGS_TO_CATEGORY',
    right_on='PRODUCT_CATEGORY',
    how='left'
)

merged_data = offer_retailer_df.merge(
    merged_brand_category,
    on='BRAND',
    how='left'
)

merged_data.drop(columns=['CATEGORY_ID', 'BRAND_BELONGS_TO_CATEGORY', 'RECEIPTS'], inplace=True)
merged_data.rename(columns={
    'PRODUCT_CATEGORY': 'BRAND_CATEGORY',
    'IS_CHILD_CATEGORY_TO': 'PARENT_CATEGORY'
}, inplace=True)

merged_data.to_csv('merged_data.csv', index=False)


In [9]:
merged_data_complete = merged_data

In [4]:
merged_data.isnull().sum()

OFFER                0
RETAILER           428
BRAND                0
BRAND_CATEGORY      50
PARENT_CATEGORY     50
dtype: int64

In [10]:
import re

def clean_text_all_special_chars(text):

    # Removing all special characters, punctuation, and symbols except alphanumeric and space
    cleaned_text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    
    return cleaned_text

merged_data_complete['CLEANED_OFFER_ALL_CHARS'] = merged_data_complete['OFFER'].apply(clean_text_all_special_chars)

merged_data_complete[['OFFER', 'CLEANED_OFFER_ALL_CHARS']].head()

,OFFER,CLEANED_OFFER_ALL_CHARS
0,Spend $50 on a Full-Priced new Club Membership,Spend 50 on a FullPriced new Club Membership
1,"Beyond Meat® Plant-Based products, spend $25",Beyond Meat PlantBased products spend 25
2,"Beyond Meat® Plant-Based products, spend $25",Beyond Meat PlantBased products spend 25
3,"Beyond Meat® Plant-Based products, spend $25",Beyond Meat PlantBased products spend 25
4,Good Humor Viennetta Frozen Vanilla Cake,Good Humor Viennetta Frozen Vanilla Cake


In [11]:
def search_offers_direct(query, df=merged_data_complete):
    query = query.lower()
    category_matches = df[(df['BRAND_CATEGORY'].str.lower() == query) | (df['PARENT_CATEGORY'].str.lower() == query)]
    brand_matches = df[df['BRAND'].str.lower() == query]
    retailer_matches = df[df['RETAILER'].str.lower() == query]
    
    # Concatenating
    all_matches = pd.concat([category_matches, brand_matches, retailer_matches]).drop_duplicates()
    
    return all_matches

search_results = search_offers_direct("BEYOND MEAT")
search_results.head()


,OFFER,RETAILER,BRAND,BRAND_CATEGORY,PARENT_CATEGORY,CLEANED_OFFER_ALL_CHARS
1,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT,Plant-Based Meat,Meat & Seafood,Beyond Meat PlantBased products spend 25
2,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT,Frozen Plant-Based Meat,Frozen,Beyond Meat PlantBased products spend 25
3,"Beyond Meat® Plant-Based products, spend $25",NaN,BEYOND MEAT,Packaged Meat,Pantry,Beyond Meat PlantBased products spend 25
47,"Beyond Steak™ Plant-Based seared tips, 10 ounc...",TARGET,BEYOND MEAT,Plant-Based Meat,Meat & Seafood,Beyond Steak PlantBased seared tips 10 ounce a...
48,"Beyond Steak™ Plant-Based seared tips, 10 ounc...",TARGET,BEYOND MEAT,Frozen Plant-Based Meat,Frozen,Beyond Steak PlantBased seared tips 10 ounce a...


# TF-IDF plus Cosine Similarity

In [64]:
def search_offers_enhanced(query, df=merged_data_complete, top_n=5):
    """
    Enhanced search function to find the most relevant offers based on the user's query.
    It performs direct matching and calculates similarity scores.
    """
    query = query.lower()
    
    # Direct Matching
    direct_matches = df[
        (df['BRAND'].str.lower() == query) | 
        (df['BRAND_CATEGORY'].str.lower() == query) | 
        (df['PARENT_CATEGORY'].str.lower() == query) | 
        (df['RETAILER'].str.lower() == query)]
    
    # Assigning a similarity score of 1 for direct matches
    direct_matches = direct_matches.copy()
    direct_matches['similarity_score'] = 1
    
    # Excluding direct matches from the similarity scoring process
    df_remaining = df.drop(index=direct_matches.index)
    
    # TF-IDF Vectorization and Cosine Similarity Calculation
    vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
    tfidf_matrix = vectorizer.fit_transform(df_remaining['OFFER'])
    query_vector = vectorizer.transform([query])
    
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    top_offers_indices = cosine_similarities.argsort()[-top_n:][::-1]
    
    similarity_matches = df_remaining.iloc[top_offers_indices]
    similarity_matches = similarity_matches.copy()
    similarity_matches['similarity_score'] = cosine_similarities[top_offers_indices]
    
    # Combining direct matches and top similarity matches
    all_matches = pd.concat([direct_matches, similarity_matches])
    
    return all_matches

# Testing the enhanced search function
search_results_enhanced = search_offers_enhanced("walmart pasta")
search_results_enhanced[['OFFER', 'BRAND', 'BRAND_CATEGORY', 'RETAILER', 'similarity_score']]


,OFFER,BRAND,BRAND_CATEGORY,RETAILER,similarity_score
776,"Barilla® pasta, select varieties, buy 2",BARILLA,Red Pasta Sauce,NaN,0.46422
549,"Barilla® pasta, select varieties, buy 3",BARILLA,Dry Pasta,NaN,0.46422
703,"Barilla® pasta, select varieties, buy 4",BARILLA,Dry Pasta,NaN,0.46422
706,"Barilla® pasta, select varieties, buy 4",BARILLA,Pesto Sauce,NaN,0.46422
777,"Barilla® pasta, select varieties, buy 2",BARILLA,Pesto Sauce,NaN,0.46422


# BERT ON OFFER

In [26]:
from transformers import DistilBertTokenizer, DistilBertModel
import torch

# Load pre-trained DistilBERT tokenizer and model
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

def generate_embedding(text):
    """
    Generate DistilBERT embedding for a given text.
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)

def search_offers_bert(query, df=merged_data_complete, top_n=5):
    """
    BERT-enhanced search function to find the most relevant offers based on the user's query.
    It generates embeddings using DistilBERT and calculates similarity scores.
    """
    query_embedding = generate_embedding(query)
    
    # Generating embeddings for the offers if not already available
    if 'embedding' not in df.columns:
        df['embedding'] = df['OFFER'].apply(generate_embedding)
    
    # Calculating cosine similarities between the query and offer embeddings
    cosine_similarities = df['embedding'].apply(lambda emb: torch.nn.functional.cosine_similarity(query_embedding, emb).item())
    
    # Getting the top N most similar offers
    top_offers_indices = cosine_similarities.nlargest(top_n).index
    top_matches = df.loc[top_offers_indices].copy()
    top_matches['similarity_score'] = cosine_similarities[top_offers_indices]
    
    return top_matches[['OFFER', 'BRAND', 'BRAND_CATEGORY', 'RETAILER', 'similarity_score']]

# Testing the BERT-enhanced search function
search_results_bert = search_offers_bert("Beyond Meat")
search_results_bert


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


,OFFER,BRAND,BRAND_CATEGORY,RETAILER,similarity_score
820,Sargento Product,SARGENTO,Cheese,NaN,0.759987
612,Any Randalls receipt,RANDALLS,NaN,RANDALLS FOOD MARKETS,0.752618
662,Any Randalls receipt,RANDALLS,NaN,RANDALLS,0.752618
716,Any Pavilions receipt,PAVILIONS,Cooking & Baking,PAVILIONS,0.746246
107,Any Vons receipt,VONS,Spirits,VONS,0.745282


In [37]:
import numpy as np
from fuzzywuzzy import fuzz

def sigmoid(x):
    """
    Sigmoid function to normalize values between 0 and 1.
    """
    return 1 / (1 + np.exp(-x))

def search_offers_normalized_similarity(query, df=merged_data_complete, top_n=5):
    """
    Search function with normalized similarity scores between 0 and 1.
    It combines direct matching, fuzzy matching, and TF-IDF similarity scoring.
    """
    query = query.lower()
    
    # Direct and Fuzzy Matching
    scores = df.apply(lambda row: max(
        fuzz.token_set_ratio(query, str(row['BRAND']).lower()) if pd.notna(row['BRAND']) else 0,
        fuzz.token_set_ratio(query, str(row['BRAND_CATEGORY']).lower()) if pd.notna(row['BRAND_CATEGORY']) else 0,
        fuzz.token_set_ratio(query, str(row['PARENT_CATEGORY']).lower()) if pd.notna(row['PARENT_CATEGORY']) else 0,
        fuzz.token_set_ratio(query, str(row['RETAILER']).lower()) if pd.notna(row['RETAILER']) else 0,
    ), axis=1)
    
    # TF-IDF Vectorization and Cosine Similarity Calculation
    vectorizer = TfidfVectorizer(stop_words='english', lowercase=True)
    tfidf_matrix = vectorizer.fit_transform(df['OFFER'])
    query_vector = vectorizer.transform([query])
    cosine_similarities = cosine_similarity(query_vector, tfidf_matrix).flatten()
    
    # Combining Scores and Applying Sigmoid Normalization
    combined_scores = scores + cosine_similarities * 100  # Giving more weight to textual similarity
    normalized_similarity_scores = sigmoid(combined_scores)
    
    # Getting the Top N Matches
    top_offers_indices = normalized_similarity_scores.nlargest(top_n).index
    top_matches = df.loc[top_offers_indices].copy()
    top_matches['similarity_score'] = normalized_similarity_scores[top_offers_indices]
    
    return top_matches[['OFFER', 'BRAND', 'BRAND_CATEGORY', 'RETAILER', 'similarity_score']]

# Testing the function with normalized similarity scores
search_results_normalized_similarity = search_offers_normalized_similarity("")
search_results_normalized_similarity


,OFFER,BRAND,BRAND_CATEGORY,RETAILER,similarity_score
88,"TWIX®, select sizes",TWIX,Candy,NaN,1.0
89,"TWIX®, select sizes",TWIX,Cookies,NaN,1.0
486,"TWIX®, select varieties",TWIX,Candy,NaN,1.0
487,"TWIX®, select varieties",TWIX,Cookies,NaN,1.0
225,"Purex® laundry detergent, select varieties, at...",PUREX,Laundry Supplies,WALMART,1.0


In [59]:
import pandas as pd
from transformers import pipeline, AutoTokenizer, AutoModel
from scipy.spatial.distance import cosine

# NER Function using BERT
def identify_entities(query):
    nlp_ner = pipeline("ner", grouped_entities=True)
    ner_results = nlp_ner(query)
    return ner_results

# Tokenization and Embedding Function
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModel.from_pretrained("bert-base-uncased")

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    outputs = model(**inputs)
    return outputs.last_hidden_state.mean(dim=1)


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [1]:
# import spacy
# from fuzzywuzzy import fuzz

# nlp = spacy.load("en_core_web_sm")

# # Function for identifying entities using spaCy
# def identify_entities_spacy(query):
#     doc = nlp(query.lower())  # Converting query to lowercase
#     entities = [ent.text for ent in doc]  # Just extracting entities as keywords
#     return entities

# # Enhanced search function using spaCy for NER
# def enhanced_search_spacy(query, df=merged_data_complete):
#     entities = identify_entities_spacy(query)
#     results = pd.DataFrame()
    
#     if not entities:
#         print("No entities recognized in the query.")
#         return results
    
#     for entity_text in entities:
#         scores = df.apply(lambda row: fuzz.token_set_ratio(entity_text, str(row['OFFER']).lower()), axis=1)
#         df['similarity_score'] = scores
        
#         top_matches = df.nlargest(5, 'similarity_score')
#         results = pd.concat([results, top_matches])
    
#     if results.empty:
#         print("No matching offers found.")
#     else:
#         return results[['OFFER', 'BRAND', 'BRAND_CATEGORY', 'RETAILER', 'similarity_score']]

# # Testing the enhanced search function using spaCy
# results = enhanced_search_spacy("WALMART")
# if not results.empty:
#     print(results)


In [13]:
import spacy
from fuzzywuzzy import fuzz

nlp = spacy.load("en_core_web_sm")

# Enhanced search function treating the entire query as a single entity
def enhanced_search_spacy(query, df=merged_data_complete):
    results = pd.DataFrame()
    
    # Concatenating relevant columns to compare against the query
    df['concatenated_text'] = df[['OFFER', 'BRAND', 'BRAND_CATEGORY', 'RETAILER']].apply(lambda row: ' '.join(row.dropna()), axis=1)
    
    # Calculating similarity scores between the query and concatenated text
    scores = df.apply(lambda row: fuzz.token_set_ratio(query.lower(), str(row['concatenated_text']).lower()) / 100, axis=1)
    df['similarity_score'] = scores
    
    # Getting the top matches based on similarity score
    top_matches = df.nlargest(100, 'similarity_score')
    results = pd.concat([results, top_matches])
    
    if results.empty:
        print("No matching offers found.")
    else:
        return results[['OFFER', 'BRAND', 'BRAND_CATEGORY', 'RETAILER', 'similarity_score']]

# Testing the enhanced search function
results = enhanced_search_spacy("walmart target")
if not results.empty:
    print(results)


                                                 OFFER  \
153  Back to the Roots Grow Seed Starting Pots OR G...   
283  Durex® Massage & Play 2 in 1 Pleasure Gel, sel...   
284  Durex® Massage & Play 2 in 1 Pleasure Gel, sel...   
484  Colgate® Toothpaste AND Colgate® Toothbrush, s...   
808  Back to the Roots Organic 3-In-1 Seed Starting...   
..                                                 ...   
157  Dove Hand Wash, select varieties, buy 2 at Target   
397  Welch's® Juicefuls® Juicy Fruit Snacks, 14 cou...   
428      L'Oréal Paris Excellence Hair Color at Target   
470  Back to the Roots Raised Bed Gardening Kit wit...   
485  L'Oréal Paris Hair color, select varieties, sp...   

                       BRAND            BRAND_CATEGORY RETAILER  \
153        BACK TO THE ROOTS    Packaged Meals & Sides   TARGET   
283                    DUREX    Medicines & Treatments   TARGET   
284                    DUREX             Sexual Health   TARGET   
484     COLGATE TOOTHBRUSHES       

In [119]:
pd.set_option('display.max_colwidth', None)
results

,OFFER,BRAND,BRAND_CATEGORY,RETAILER,similarity_score
625,Tostitos® Toppers™,TOSTITOS,Dips & Salsa,NaN,0.39
114,Spend $150 at Vons,VONS,Spirits,VONS,0.35
179,Spend $100 at Vons,VONS,Spirits,VONS,0.35
489,Spend $275 at Vons,VONS,Spirits,VONS,0.35
663,Spend $75 at Randalls,RANDALLS,NaN,RANDALLS,0.34
...,...,...,...,...,...
586,"GOYA® Adobo Seasoning, 8 ounce+",GOYA,Pickled Goods,NaN,0.24
589,"GOYA® Adobo Seasoning, 8 ounce+",GOYA,Deli Counter,NaN,0.24
682,Spend $10 at CVS,CVS,Skin Care,CVS,0.24
713,"Arber, at Walmart",ARBER,NaN,WALMART,0.24


In [14]:
!pip freeze

absl-py @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_e3n1cffagz/croot/absl-py_1666362938899/work
aiohttp @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/abs_4c_8pz93lf/croot/aiohttp_1670009562783/work
aiosignal @ file:///tmp/build/80754af9/aiosignal_1637843061372/work
alabaster @ file:///home/ktietz/src/ci/alabaster_1611921544520/work
anaconda-client @ file:///Users/builder/miniconda3/envs/prefect/conda-bld/anaconda-client_1635406085191/work
anaconda-navigator==2.4.0
anaconda-project @ file:///tmp/build/80754af9/anaconda-project_1637161053845/work
anyio @ file:///private/var/folders/nz/j6p8yfhx1mv_0grj5xl4650h0000gp/T/croot-t_zs64wy/anyio_1644482593257/work/dist
appdirs==1.4.4
appnope @ file:///Users/ktietz/demo/mc3/conda-bld/appnope_1629146036738/work
appscript @ file:///Users/ktietz/demo/mc3/conda-bld/appscript_1629302122760/work
argon2-cffi @ file:///opt/conda/conda-bld/argon2-cffi_1645000214183/work
argon2-cffi-bindings @ file:///private/v